# Sentiment Analysis

*Sentiment analysis* is the task of evaluating whether a given passage of text is primarily "positive" or "negative." The meanings of these terms can change in context. For example, a "positive" product review would indicate that the customer likes the product, whereas a "positive" tweet might just indicate that the user is happy that day. 

In this lecture, we'll discuss how familiar machine learning tools can allow us to perform sentiment analysis on unstructured text. 

Our data set for this task comes from the `nltk` package again. It's a set of movie reviews. 

In [1]:
#standard imports 
import numpy as np
import pandas as pd

In [2]:
#use nltk to get movie reviews
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


The `movie_reviews` object allows us to read in the data. 

In [3]:
movie_reviews

<CategorizedPlaintextCorpusReader in 'C:\\Users\\micha\\AppData\\Roaming\\nltk_data\\corpora\\movie_reviews'>

For today, the two most important methods of this object are `fileids()` and `raw()`. The first method will allow us to locate the files on disk in which the movie reviews are contained, and the second method will allow us to then obtain the full text of the reviews from the file path. 

Let's first look at the fileids. 

In [4]:
type(movie_reviews.fileids())

list

In [5]:
review_list=movie_reviews.fileids()
len(review_list)

2000

movie_reviews.fileids() returns a list of 2000 fileids

In [6]:
print(review_list[0],review_list[1000])

neg/cv000_29416.txt pos/cv000_29590.txt


The first 1000 are in a folder called neg and the last 1000 are in a file called pos.

We can use moview_reviews raw method to read the reviews contained in the files.

In [7]:
#movie_reviews.raw(review_list[0])

### How can we read in the complete dataset???

One option would be to use for loops, but this will be slow and won't give us the data  in a nice format. A betters solution is to use dataframes and the apply method.

In [8]:
df=pd.DataFrame({"fileid":review_list})
df.head(3)

,fileid
0,neg/cv000_29416.txt
1,neg/cv001_19502.txt
2,neg/cv002_17424.txt


So far we have a dataframe with one column, the fileids. Now, let's create another column by __applying__ the raw method to the fileid column

In [9]:
df['raw_text']=df['fileid'].apply(movie_reviews.raw)
df.head()

,fileid,raw_text
0,neg/cv000_29416.txt,"plot : two teen couples go to a church party ,..."
1,neg/cv001_19502.txt,the happy bastard's quick movie review \ndamn ...
2,neg/cv002_17424.txt,it is movies like these that make a jaded movi...
3,neg/cv003_12683.txt,""" quest for camelot "" is warner bros . ' firs..."
4,neg/cv004_12641.txt,synopsis : a mentally unstable man undergoing ...


We can use tools from before to create a term-document matrix. This time, we treat each movie review as a document. 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer(max_df=.2,min_df=30,stop_words="english")
counts=vec.fit_transform(df['raw_text'])

In [11]:
counts_df=pd.DataFrame(counts.toarray(),columns=vec.get_feature_names())
counts_df.head(3)

,000,10,100,11,12,13,15,17,18,1993,...,writing,written,wrong,wrote,yeah,yes,york,younger,youth,zero
0,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [12]:
df=pd.concat((df,counts_df),axis=1)
df.head(3)

,fileid,raw_text,000,10,100,11,12,13,15,17,...,writing,written,wrong,wrote,yeah,yes,york,younger,youth,zero
0,neg/cv000_29416.txt,"plot : two teen couples go to a church party ,...",0,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,neg/cv001_19502.txt,the happy bastard's quick movie review \ndamn ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,neg/cv002_17424.txt,it is movies like these that make a jaded movi...,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


Now, let's add a column called 'is_good' which captures if the review is positive or negative

In [13]:
df['is_good']=df['fileid'].str.split('/').str.get(0)>="p"

We have now successfully read in and prepared our data. 

---

# On to Sentiment Analysis

These steps should be pretty familiar. We are going to split our data into training and test sets, create a logistic classifier, and evaluate the logistic classifier on the test set.

As usual, we start by splitting into train and test

In [14]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=.4)

Now, let's select the appropriate columns. Our target is the 'is_good' column. From our training set, let's exclude the fileid, raw_text, and is_good

In [15]:
X_train=train.drop(['fileid','raw_text','is_good'],axis=1)
y_train=train['is_good']

X_test=test.drop(['fileid','raw_text','is_good'],axis=1)
y_test=test['is_good']

Now, lets apply a Logitic Regression Model

In [16]:
from sklearn.linear_model import LogisticRegression

LR=LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [17]:
LR.score(X_train,y_train)

1.0

Is this too good to be true???

In [18]:
from sklearn.model_selection import cross_val_score
cross_val_score(LR,X_train,y_train,cv=5).mean()

0.7699999999999999

Our model perfectly fits the test data, but based on CV it looks like our predictive accuracy might only be around 80%. This looks like overfitting, which makes sense -- overfitting is a very common problem when we have many predictor columns (lots of words) and not that many data observations. 

There are multiple ways to address this. In this lecture, let's use cross-validation to tune the regularization parameter

In [19]:
C_pool=np.linspace(.005,.05,10)
best_score=-np.inf

for c in C_pool:
    LR=LogisticRegression(C=c)
    score=cross_val_score(LR,X_train,y_train,cv=5).mean()
    if score>best_score:
        best_score=score
        best_c=c
    print("C=",np.round(c,3)," Cross validation score=",score)

C= 0.005  Cross validation score= 0.8083333333333333
C= 0.01  Cross validation score= 0.8091666666666667
C= 0.015  Cross validation score= 0.8033333333333333
C= 0.02  Cross validation score= 0.7991666666666667
C= 0.025  Cross validation score= 0.7975
C= 0.03  Cross validation score= 0.7975
C= 0.035  Cross validation score= 0.7941666666666667
C= 0.04  Cross validation score= 0.7925000000000001
C= 0.045  Cross validation score= 0.7925000000000001
C= 0.05  Cross validation score= 0.7908333333333333


Now, let's use the best c to evaluation on the test set

In [20]:
best_c

0.010000000000000002

In [21]:
LR=LogisticRegression(C=best_c)
LR.fit(X_train,y_train)
LR.score(X_test,y_test)

0.80625

As we can see, using the best_c increased our accuracy! 

__However, we're not done yet.__ 

One of the primary purposes of sentiment analysis is to determine which words carry positive or negative associations. It is common to assign scores to each word that govern how positive or negative they are. We can do this using the coefficients of the logistic model. First, let's make a data frame of the words and their scores. 

In [22]:
sentiment_df=pd.DataFrame({"word":X_train.columns,"coef":LR.coef_[0]})
sentiment_df

,word,coef
0,000,-0.045913
1,10,0.022241
2,100,-0.002140
3,11,-0.006792
4,12,-0.024459
...,...,...
3019,yes,-0.021197
3020,york,0.010750
3021,younger,0.023898
3022,youth,0.022019


Now, let's look at the most negative words

In [23]:
sentiment_df.sort_values('coef',ascending=True).head(10)

,word,coef
308,boring,-0.203416
3002,worst,-0.198800
2587,stupid,-0.155068
2626,supposed,-0.152149
2837,unfortunately,-0.129123
1993,poor,-0.128187
217,awful,-0.124672
2881,video,-0.124226
1731,minute,-0.124111
2224,ridiculous,-0.123214


What about postive words?

In [24]:
sentiment_df.sort_values('coef',ascending=False).head(10)

,word,coef
1938,performances,0.173507
1282,hilarious,0.141971
925,excellent,0.126062
2693,terrific,0.114589
745,different,0.112207
1936,perfectly,0.111407
1163,gives,0.110231
989,family,0.106135
250,beautiful,0.104455
2803,truman,0.103557


This also looks pretty logical. We can conclude that our model has had some success in learning which words have positive and negative meanings. 

Of course, the story isn't over: there are many different models that can be used for sentiment analysis, some of which highlight different features. 

Finally, the combination of term-document extraction with classification models isn't just for sentiment analysis! Essentially the same pipeline can work to produce a functioning spam classifier, in which a "negative" set of text is spam and a "positive" set of text is a legitimate email. 